In [1]:
import pandas as pd
import numpy as np

# Préparation des inputs

Chargement des données

In [2]:
data = pd.read_csv('data_processed')

Ajout des variables mois et jours de la semaine

In [3]:
data['date'] = data['date'].apply(lambda x : pd.Timestamp(x))
data['month'] = data['date'].apply(lambda x : x.month)
data['weekday'] = data['date'].apply(lambda x : x.weekday())

One hot encoding pour les varialbes jours de la semaine, mois et CBN

In [4]:
data['CBN'] = pd.Categorical(data['CBN'])
data['month'] = pd.Categorical(data['month'])
data['weekday'] = pd.Categorical(data['weekday'])

# Créer les one hot encodings pour chaque variabble
dfDummies_cbn = pd.get_dummies(data['CBN'], prefix = 'CBN')
dfDummies_month = pd.get_dummies(data['month'], prefix = 'month')
dfDummies_weekday = pd.get_dummies(data['weekday'], prefix = 'weekday')

# Concatener les one hot encodings avec les données
data = pd.concat([data, dfDummies_weekday, dfDummies_month, dfDummies_cbn], axis=1)

# Suppression de la variable mois, jour de la semaine et CBN
data.drop(columns=['CBN', 'month', 'weekday'], inplace=True)

In [6]:
data.head()

,date,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,month_1,month_2,...,CBN_X 06,CBN_X 07,CBN_X 08,CBN_X 09,CBN_X 10,CBN_X 11,CBN_X 12,CBN_Y,CBN_Z,CBN_j 04
0,2019-03-01 06:09:37,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019-03-01 06:09:40,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2019-03-01 06:09:45,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019-03-01 06:09:48,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2019-03-01 06:09:51,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# LSTM